In [1]:
!pip install langchain neo4j langchain-openai langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00


In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from dotenv import load_dotenv

load_dotenv("/content/.env")

# 실제 인스턴스 정보를 입력합니다.
NEO4J_URI = "bolt://localhost"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "password"
NEO4J_DATABASE = "neo4j"

embedding = OpenAIEmbeddings()

# Neo4j 그래프 객체 생성
graph = Neo4jVector.from_existing_graph(
    embedding=embedding,
    node_label="__Entity__",
    text_node_properties=["description"],
    embedding_node_property="embedding",
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)

# Neo4jGraph 객체 추가 생성 (Cypher 쿼리 실행용)
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

In [ ]:
def fetch_entity_context(entity_name):
    context = {"name": entity_name}
    try:
        # 텍스트 청크 가져오기
        chunk_query = """
        MATCH (e:__Entity__ {name: $entity_name})<-[:HAS_ENTITY]-(c:__Chunk__)
        RETURN c.text AS text
        """
        chunk_result = neo4j_graph.query(chunk_query, {"entity_name": entity_name})
        context["text_chunks"] = [r["text"] for r in chunk_result] if chunk_result else ["No text chunk available"]

        # 커뮤니티 보고서 가져오기
        community_query = """
        MATCH (e:__Entity__ {name: $entity_name})-[:IN_COMMUNITY]->(com:__Community__)
        RETURN com.full_content AS report
        """
        community_result = neo4j_graph.query(community_query, {"entity_name": entity_name})
        context["community_reports"] = [r["report"] for r in community_result] if community_result else ["No community report available"]

        # 관련 엔티티 가져오기
        related_query = """
        MATCH (e:__Entity__ {name: $entity_name})-[:RELATED]->(related:__Entity__)
        RETURN related.name AS name, related.description AS description
        """
        related_result = neo4j_graph.query(related_query, {"entity_name": entity_name})
        context["related_entities"] = (
            [{"name": r["name"], "description": r["description"]} for r in related_result]
            if related_result else []
        )
    except Exception as e:
        context["error"] = f"Error fetching context: {str(e)}"
    return context

In [ ]:
def create_structured_context(all_contexts, query):
    context_str = "## 질문과 관련된 엔티티 정보\n\n"
    context_str += "아래는 질문에 답변하는 데 유용한 엔티티들의 구조화된 정보입니다:\n\n"

    for i, ctx in enumerate(all_contexts, 1):
        context_str += f"### 엔티티 {i}: {ctx['name']}\n"
        context_str += f"- **설명**: {ctx['description']}\n"
        context_str += "- **텍스트 청크**:\n"
        for chunk in ctx['text_chunks']:
            context_str += f"  - {chunk}\n"
        context_str += "- **커뮤니티 보고서**:\n"
        for report in ctx['community_reports']:
            context_str += f"  - {report}\n"
        if ctx['related_entities']:
            context_str += "- **관련 엔티티**:\n"
            for rel in ctx['related_entities']:
                context_str += f"  - {rel['name']}: {rel['description']}\n"
        else:
            context_str += "- **관련 엔티티**: 없음\n"
        context_str += "\n"
    return context_str

In [ ]:
# LLM 설정 (예: GPT-4o)
llm = ChatOpenAI(model="gpt-4o")

# 리트리버 설정
retriever = graph.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# 질문 설정
query = "마일당 순이익(NET INCOME PER MILE)을 어떻게 분석해야 하나요?"
results = retriever.get_relevant_documents(query)

# 모든 엔티티의 컨텍스트 수집
all_contexts = []
for result in results:
    entity_name = result.metadata.get("name", "Unknown")
    description = result.page_content
    context = fetch_entity_context(entity_name)
    context["name"] = entity_name
    context["description"] = description
    all_contexts.append(context)

context_str = create_structured_context(all_contexts, query)
prompt = f"아래 맥락에 기반해서, 주어진 질문에 한국어로 답하세요\n\n**질문**: {query}\n\n**맥락**:\n{context_str}"
response = llm.invoke(prompt)
print("Final Response:")
print(response.content)

Final Response:
마일당 순이익(NET INCOME PER MILE)을 분석할 때 다음의 주요 요소들을 고려해야 합니다:

1. **순이익 계산**: 순이익은 총 수익에서 운영 비용(때로는 세금)을 뺀 다음, 다른 소득원을 추가하여 계산됩니다. 이는 철도의 수익성 및 재무 건전성을 평가하는 중요한 척도입니다.

2. **운영 비용 분석**: 운영 비용은 총 수익의 약 65%를 차지하는 것이 일반적이며, 이는 철도의 재정 관리와 효율성을 평가하는 데 중요한 역할을 합니다. 운영 비용과 총 수익의 균형을 이해하면 철도의 재정 관리 관행 및 수익성 잠재력을 평가하는 데 유용합니다.

3. **성장 추세 확인**: 보고서를 통해 순이익이 증가하고 있는지 아니면 감소하고 있는지를 파악해야 합니다. 이것은 철도의 운영 효율성을 평가하는 데 중요한 정보입니다.

4. **비교 분석**: 같은 분야의 다른 철도와의 순이익 비교 분석을 통해 해당 철도의 상대적 위치와 경쟁력을 평가할 수 있습니다.

5. **고정비 비중**: 순이익과 고정비(예: 채권 이자 및 세금) 간의 비율을 파악하여 철도의 재무 안정성을 평가할 수 있습니다. 일반적으로 산업 채권이 호의적으로 평가받기 위해서는 연간 순이익이 연간 채권 이자, 세금 및 저당 기금의 약 세 배가 되어야 합니다.

6. **투자 및 유지보수**: 철도의 주요 설비, 노후된 자산들의 유지보수, 자본 집행 투자 등이 적절하게 이루어지고 있는지 확인하여 철도의 장기적 안정성을 평가하는 것이 중요합니다.

이러한 요소를 분석하여 마일당 순이익이 철도의 효율성 및 재무 건전성을 어떻게 나타내는지 이해할 수 있습니다. 이를 통해 투자자들은 철도에 대한 투자 결정을 내리는 데 유용한 정보를 얻을 수 있습니다.


Global Retriever

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [ ]:
MAP_SYSTEM_PROMPT = """
---역할---
제공된 컨텍스트를 참고하여 사용자의 질문에 답하는 어시스턴트입니다.

---목표---
질문과 관련된 컨텍스트 정보를 요약한 주요 포인트 목록을 JSON 형식으로 생성하세요.
정보가 부족하면 "모르겠습니다"라고 답하세요.

각 포인트는 다음을 포함해야 합니다:
- 설명: 포인트에 대한 상세 설명.
- 중요도 점수: 0~100 사이의 정수.

데이터 참조 예:
"예시 문장 [Data: Reports (2, 7, 64, 46, 34, +more)]"
(한 참조에 5개 이상의 id는 "+more"를 사용)

출력 예:
{{
    "points": [
        {{"description": "포인트 1 설명 [Data: Reports (보고서 id들)]", "score": 점수}},
        {{"description": "포인트 2 설명 [Data: Reports (보고서 id들)]", "score": 점수}}
    ]
}}
"""


map_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", MAP_SYSTEM_PROMPT),
        ("human", "question: {question}\n\n context: {context}"),
    ]
)

map_chain = map_prompt | llm | StrOutputParser()


In [29]:
REDUCE_SYSTEM_PROMPT = """
---역할---
여러 분석가의 보고서를 종합하여 사용자의 질문에 답하는 어시스턴트입니다.

---목표---
제공된 분석가 보고서를 바탕으로, 질문에 대한 종합적인 답변을 마크다운 형식으로 작성하세요.
불필요한 정보를 제거하고, 핵심 포인트와 시사점을 포함하세요.
정보가 부족하면 "모르겠습니다"라고 답하세요.

---분석가 보고서---
{report_data}

데이터 참조는 다음 형식을 유지하세요:
"예시 문장 [Data: Reports (2, 7, 34, 46, 64, +more)]"
(한 참조에 5개 이상의 id는 "+more"를 사용)

대상 응답 길이 및 형식: {response_type}
"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", REDUCE_SYSTEM_PROMPT),
        ("human", "{question}"),
    ]
)

reduce_chain = reduce_prompt | llm | StrOutputParser()


In [30]:
response_type: str = "multiple paragraphs"


def global_retriever(query: str, level: int, response_type: str = response_type) -> str:
    community_data = graph.query(
        """
    MATCH (c:__Community__)
    WHERE c.level = $level
    RETURN c.full_content AS output
    """,
        params={"level": level},
    )
    intermediate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermediate_response = map_chain.invoke(
            {"question": query, "context": community["output"]}
        )
        intermediate_results.append(intermediate_response)
    final_response = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_response

In [32]:
from tqdm import tqdm
print(global_retriever("이 책의 주제가 뭐야?", 1))

> <ipython-input-30-fa14bd32d2f6>(22)global_retriever()
     21     ipdb.set_trace()
---> 22     for community in tqdm(community_data, desc="Processing communities"):
     23         intermediate_response = map_chain.invoke(

ipdb> c


Processing communities: 100%|██████████| 22/22 [02:46<00:00,  7.57s/it]


> <ipython-input-30-fa14bd32d2f6>(28)global_retriever()
     27     ipdb.set_trace()
---> 28     final_response = reduce_chain.invoke(
     29         {

ipdb> c
이 책은 금융 생태계 내에서 다양한 금융 자산 간의 상호작용과 경제 주기의 영향을 탐구하고 있습니다. 주로 채권, 주식, 경기침체에 집중하며, 투자와 관련된 다양한 전략과 시장 영향을 다룹니다. 경제적 사이클과 금리 변동이 채권 및 주식의 가치 평가에 미치는 영향을 비롯하여, 사업 상황과 경제적 조건이 중등급 및 저등급 채권과 주식의 성과에 미치는 영향을 설명합니다.

금리가 경제 활동과 투자 결정에 중심적인 역할을 하며, 이러한 금리 변동의 영향력이 채권 시장, 특히 고급 채권과 선택된 지방채, 그리고 2급 철도 발행물에서 어떻게 나타나는지를 분석합니다. 또한, 투자자들이 금리와 신용 사이클에 대한 주의를 기울여야 함을 강조하고 있습니다. 채권, 특히 저등급 채권은 금리 변화에 민감하며, 이에 따라 투자자들은 금리 및 신용 사이클을 면밀히 모니터링해야 합니다.

경제적 사이클 또한 투자 결정에 있어 중요하며, 유동 자본의 흐름과 경제 번영 및 침체 사이의 전환이 새 사업을 촉진하는 방식에 대해서도 논의합니다. 이러한 경제 주기가 금융 안정성에 미치는 영향을 통해 정책 입안자와 투자자들이 경제 변동에 대비할 수 있도록 도움을 제공합니다.

[Data: Reports (53, 59, 70, 116, 124, 125, +more)]
